MAIN PROGETTO DI DEEP LEARNING:
questo lo andremo ad analizzare punto per punto e louserò nel progetto...spero.

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd 
import os 

Oggetto che modella il dataset di training e di test.
Il dataset di tr e te, servirà ad avere un riferimento a dove si trovano le immagini o i dati in generale.
Ci offrirà funzioni per attingere a queste immagini /dati 

-I dataset sono gli oggetti che ci farano perdere più tempo in fase di implementazione, perchè difficilmente si ricicla da un progetto ad un altro. 
-In questa eseritazione utilizzeremeo dei dataset già presenti in torchvision e ma in generale dovrebbe essere più complicato.

In [ ]:
training_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

In [ ]:
test_dataset = datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

-Dataloader: si preoccupa di campionare le batch dal dataset e di mischiare il dataset al termine di ogni epoca. ovviamente solo durante il training.
-sono oggetti di pytorch, e li si importa da utils.data da pytorch.
-vuole un riferimento al dataset.
-la batch size, cioè quanti cmpioni deve campionare per creare la batch.
-se deve shuffleare il dataset ogni volata che cambio epoca.

per il test stessa identica cosa eccetto per qianto riguarda lo shuffle.

In [ ]:
train_dataloader = DataLoader(training_dataset, batch_size=64 , shuffle=True)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Ci serve il modello ovviamente:
non esiste la classe rete neurale, dovremo creare noi a mano la nostra rete.

-si crea una classe neural network, che deve estendere una classe base di pytorch.
-Module è una classe molto generica che modella layer reti neurali intere, è talmente generica che tutti i metodi la riconoscono.
-La prima funzione da implementare è il Costruttore:
    . Eredita il costruttore della classe Module
    . Si crea la struttura della rete. Introducendo i vari strati che la compongono.
        in questo caso avremo:
        . flatterizzatore: prende un tensore di più dimensioni e lo flatterizza
        . primo layer fc, i due parametri sono dimensionalità dell'input e nodi del layer a cui arrivo.
        . layer relu
        . secondo layer fc che prende 512 input(stesso output del primo) e sputa 10 output
-secondoa funzione da implementare: forward per il processamento della rete da input a output.

In [ ]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __int__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 512) #input + quanti nodi ha quel layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self , X):
        X  = self.flatten(X)
        X = self.fc1(X)
        X = self.relu(X)
        X = self.fc2(X)
        return X

In [ ]:
model = NeuralNetwork()

hyperparameters:

In [ ]:
learning_rate = 0.01
batch_size = 64 #campioi selezionati ad ogni iterazione dell'ottimizzatore.
epochs = 5


Loss funcion: 
-dipende dal task.
-se vogliamo fare una classificazione possiamo utilizzare cross entropy loss
-se vogliamo fare una regressione possiamo utilizzare MSE

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

Ottimizzatore:
-semplicemente li dobbiamo scegliere, dal sottopachetto .optim
-vedremo che per come costruiremo il model(), questo avrà una funzione parameters che restituisce la lista di tutti i parametri della rete.
-comunichiamo la lista, perchè nel loop ad ogni chiamata di optimizer.step() avverrà l'aggiornamento dei parametri mediante quello specifico ottimizzatore
-lr passato come parmaetro che noi abbiamo già scelto.

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

LOOP DI OTTIMIZZAZIONE: la struttura sarà verosimilmente molto simile a quella riportata nella cella sottostante. 
-> Primo loop sulle epoche.
-> Secondo loop sulle batch. leggermente più complesso, perchè vengono utilizzate dei comandi paricolari:
    - batch_indx mi dirà quante batch go processato fin'ora.
    - (X , y) quello che mi deve restituire la batch dei campioni, ovvero features ed etichette.
    - enumerate(train_dataloader) mi da qualcosa che è un iteratore, ovvero una lista di coppie (indice, valore(in questo caso una tupla)).
-> Cosa succede nel secondo loop?
    -chiedo di effettuare il forward, quindi la prediction del mio modello basandomi sui dati X
    -calcolo la loss.
    -Azzero i gradienti: operazione che va fatta perchè di default l'ottimizzatore accumula i gradienti tra più iterazioni
    -loss.Backpropagation()
    - Dopo aver calcolato tutti i gradienti lungo l'albero di calcolo, faccio fare uno step all'ottimizzatore (applica la formula adam per esempio, predi i valori dei parametri e li aggiorni con la tecnica che ti dico di usare.)

-Terminato il secondo loop, ho terminato un'epoca. A questo punto il dataloader viene resettato, internamente il dataloader ha una funzione per mischiare i dati, inizia una nuova epoca, e ricomincia tutto e da capo.

-Attenzione nessuno mi vieta al termine di ogni epoca di effettuare un test magari su di un validation set. Ricordiamoci però di usare il comando with torch.no_grad(): perchè non vogliamo che vengano calcolati i gradienti in fase di verifica (non serve)

In [ ]:
for epoch in epochs:
    for batch_indx , (X, y) in enumerate(train_dataloader):

        prediction = model(X)
        loss = loss_fn(prediction, y)

        optimizer.zero_grad()
        loss.backward() #Calcolo i gradienti 
        optimizer.step() #Modifico i parametri


        if batch_indx % 100 == 0:
            loss, current = loss.item(), batch_indx * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    correct = 0
    with torch.no_grad():
        for batch_indx , (X, y) in enumerate(test_dataloader):

            prediction = model(X) #SPIEGO SOTTO
            #loss = loss_fn(prediction, y)
            #test_loss += loss.item()
            correct += (prediction.argmax(1) == y).type(torch.float).sum().item()

    print("Accuracy: ", 100*correct/len(test_dataloader.dataset) , "%")




Spiegazione ultima parte: 
prediction essendo un porblema di classificazione mi fornisce delle probabilità per ognuna delle 3 possibili classi. A me interessa ovviamente la probabilità più alta, cioè quella che verosimilmente mi indica il risultato corretto.

-prediction.argmax(1) estraggo il valore più grande di prediction.
-faccio un check per vedere se il valore più grande è uguale al valore della classe.
-se è uguale allora aumento il numero di predizioni corrette, convertendolo a float, sommando sulla batch e estraendo il valore

Una volta fatto per tutto il dataset di test, me printo l'accuracy.

COMMENTO SUI DATASET: 
Mentre il dataloader sarà essenzialmente sempre come lo abbiamo creato sopra, perchè è un oggetto che non pspecigfico del progetto, lui ha un riferimento a ds pesca i dati e costruisce le batch. A prescindere da come è fatto il dataset. 

Il dataset è specifico per ofgni task, quindi non ho un oggetto generale.
Come faccio a creare un oggetto dataset custom? 

Sotto ci sarà un esempio:

In [ ]:
from torch.utils.data import Dataset
from torchvision.io import read_image

class MyDataset(Dataset):
    def __init__(self , annotation_file , img_dir , transform = None):
        self.img_labels = pd.read_csv(annotation_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self , indx): #Devo tirare fuori un'immagine e un'etichetta concordi.
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[indx, 0]) #--> è il nome 0000.png dell'immagine
        image = read_image(img_path)
        label = self.img_labels.iloc[indx, 1]
        if self.transform:
            image = self.transform(image)
        return image , label


E' importnte rispettare delle regole:
in particolare è importante rispettare che l'oggetto dataset che creiamo abbia un link con pytorch, dove questo link significa che la classe che andrò ad implementare e che rappresenta il mio dataset, deve ereditare una classe base di pytorch, che rappresenta un dataset genrrico. Classe astratta. Devo estendere una classe base di torch. 

-from torch.utils.data import Dataset
-creiamo la classe che eredita informazioni dalla classe Dataset, che è messa tra parentesi.
-creazione del costruttore: all'interno ci possono esssere diversi parametri questo è un esempio. Nel mio caso sto ipotizzando che il mio dataset sia una cartella di immagini persente nell'had_disk.
    -img_dir è un path ad una cartella in cui sono presenti le vere immagini.
    -annotation_file è un file in cui sono presenti tutte le etichette delle immagini. (img0001.png 0)
    -transform è una funzione che trasforma i dati.
-la funzione __len__(self) che deve ritornare sempre il numero di campioni del dataset. Come si calcola la lunghezza di questo dataset, è relativo al singolo problema. Questa informazione comunque è fondamentale, perchè questa informazione serve al dataloader per capire quante volte iterare sul dataset.

Se annotation_file è di tipo CSV, 0000.png, 1 e così via per ogni riga.
Se io ho questo file di annotazione e da un'altra parte ho una cartella con le immagini dove effettivamente c'è l'immagine 0000.png.
Allora io potrei fare pandas read csv e leggerle, esattamente come viene fatto nel codice di spora.

Quindi in questo caso self.img_labels è una lista lunga quante sono le mie immagini.

- __getitem__ è la funzione che il dataloader chiama quando deve costruire le batches. Quinidi se io chiedo una batch di 64 elementi allora lui chiamerà 64 volte la funzione getitem sul dataset, e si fa dare ogni volta un sample cioè una coppia immagine etichetta.
l'unica cosa che non mi è veramente chiara è l'indx che devo utilizzare. Però dato che questa funzione è chiamata dal dataloader, allora sarà lui a fornirgli l'index. 
Il dataloader fornisce un indx a caso nel range della len del dataset.



